# NBA Games Predictor - Basic

This notebook takes all the regular season games from the NBA 2016-17 season.  Using a series of regression & supervised learning techniques, it attempts to accurately predict the outcome of basketball games.

In [1]:
import json 
import pandas as pd
import numpy as np 
import geopy.distance

#package for flattening json in pandas df
from pandas.io.json import json_normalize

In [2]:
# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

## Configuration

In [3]:
use_parsed_data = False
output_parsed_data_filename = '../input/parsed_data.csv'

calc_last_5_games = True
calc_win_loss = True
calc_win_loss_write_all = True
calc_ratings = True
use_team_keys = False
use_team_hot_encoding = False
use_team_home_away_hot_encoding = False

## Load data from file system

In [4]:
#load games - json object
if use_parsed_data == True:
    games = pd.read_csv(output_parsed_data_filename)
    games = games.drop(['Unnamed: 0'], axis=1)
else:
    with open('../input/games.json') as gamesJsonRaw:
        d = json.load(gamesJsonRaw)

    games = json_normalize(d['games'])

In [5]:
games.sample(5)

,id,location,oddsBet365Away,oddsBet365Home,scoreAway,scoreHome,scoreQuarters,season,statsAway.Ast,statsAway.Blk,...,statsHome.Reb,statsHome.Stl,statsHome.Tov,statsHome.WinPct,statsHome.Wins,teamAwayCode,teamAwayId,teamHomeCode,teamHomeId,time
590,31767,TD Garden,4.75,1.2,103,117,"[{'number': 1, 'scoreHome': 36, 'scoreAway': 2...",2015,24,5,...,66,12,12,0.000,1,PHX,104,BOS,82,2016-01-15T19:30:00.000Z
2222,34937,Sleep Train Arena,1.90,1.9,115,120,"[{'number': 1, 'scoreHome': 24, 'scoreAway': 2...",2016,32,4,...,46,9,15,1.000,1,ORL,95,SAC,103,2017-03-13T22:30:00.000Z
817,31994,Staples Center,1.50,2.7,115,112,"[{'number': 1, 'scoreHome': 20, 'scoreAway': 3...",2015,32,2,...,53,9,14,0.000,0,GSW,101,LAC,102,2016-02-20T20:30:00.000Z
104,31280,Sleep Train Arena,1.27,3.9,106,88,"[{'number': 1, 'scoreHome': 24, 'scoreAway': 2...",2015,34,7,...,49,10,19,0.000,0,SAS,106,SAC,103,2015-11-09T22:00:00.000Z
980,32157,Philips Arena,2.70,1.5,75,104,"[{'number': 1, 'scoreHome': 30, 'scoreAway': 1...",2015,24,5,...,58,11,15,0.000,1,IND,87,ATL,91,2016-03-13T18:00:00.000Z


In [6]:
#load stadiums - json object
with open('../input/stadiums.json') as stadiumsJsonRaw:
    stadiumsJson = json.load(stadiumsJsonRaw)

stadiums = json_normalize(stadiumsJson['stadiums'])

In [7]:
stadiums.head(5)

,lat,lng,team
0,33.757183,-84.396278,ATL
1,42.366281,-71.062266,BOS
2,40.682650,-73.974689,BRO
3,35.224519,-80.841053,CHA
4,41.880589,-87.674149,CHI


   ## Data Preperation

### Record the names of all stats - to be dropped later.

In [8]:
statsColumns = pd.Series(list(games.columns.values))
statsColumns = statsColumns.loc[statsColumns.str.startswith(('statsAway', 'statsHome'), na=False)]

In [9]:
# Replace any NaN with 0 in the stats
games[statsColumns] = games[statsColumns].fillna(value=0)

### Set Valid Data Types

In [10]:
#games['season'] = games['season'].astype('float')
games['scoreAway'] = games['scoreAway'].astype('uint8')
games['scoreHome'] = games['scoreHome'].astype('uint8')
games['teamAwayId'] = games['teamAwayId'].astype('uint8')
games['teamHomeId'] = games['teamHomeId'].astype('uint8')

games['statsHome.FgAtt'] = games['statsHome.FgAtt'].astype('uint8')
games['statsHome.OffReb'] = games['statsHome.OffReb'].astype('uint8')
games['statsHome.Tov'] = games['statsHome.Tov'].astype('uint8')
games['statsHome.FtAtt'] = games['statsHome.FtAtt'].astype('uint8')
games['statsHome.Pts'] = games['statsHome.Pts'].astype('uint8')
games['statsHome.PtsAgainst'] = games['statsHome.PtsAgainst'].astype('uint8')

games['statsAway.FgAtt'] = games['statsAway.FgAtt'].astype('uint8')
games['statsAway.OffReb'] = games['statsAway.OffReb'].astype('uint8')
games['statsAway.Tov'] = games['statsAway.Tov'].astype('uint8')
games['statsAway.FtAtt'] = games['statsAway.FtAtt'].astype('uint8')
games['statsAway.Pts'] = games['statsAway.Pts'].astype('uint8')
games['statsAway.PtsAgainst'] = games['statsAway.PtsAgainst'].astype('uint8')

#scoreQuarters          object
#teamAwayCode           object
#teamHomeCode           object

### Calculate the number of games played for season

In [11]:
if (use_parsed_data == False):

    # Create a dataframe for the results - same size as dataset
    data = pd.DataFrame(index=range(0,len(games)), columns=['gamesPlayedHome', 'gamesPlayedAway'])

    # Iterate through every team
    for team in games.teamAwayCode.unique():
        season = 0

        # Iterate through each game the team is present in.
        for index, game in games[(games['teamAwayCode'] == team) | (games['teamHomeCode'] == team)].sort_values(by='season', ascending=True).iterrows():
            if season != game.season:
                gameCount = 0
                homeGameCount = 0
                season = game.season

            gameCount += 1

            # Update game count for team - whether away or home.
            if game.teamAwayCode == team:
                data.loc[index]['gamesPlayedAway'] = gameCount
            else:
                data.loc[index]['gamesPlayedHome'] = gameCount

    data['gamesPlayedAway'] = data['gamesPlayedAway'].astype('uint8')
    data['gamesPlayedHome'] = data['gamesPlayedHome'].astype('uint8')

    # Append the results to the dataset
    games = pd.merge(games, data, left_index=True, right_index=True)

In [12]:
games[['teamHomeCode', 'teamAwayCode', 'season', 'gamesPlayedHome', 'gamesPlayedAway']].sample(5)

,teamHomeCode,teamAwayCode,season,gamesPlayedHome,gamesPlayedAway
2436,GSW,UTA,2016,59,59
407,PHX,MIL,2015,81,81
1169,WAS,BRO,2015,26,26
1981,WAS,NOP,2016,10,10
874,ATL,CHA,2015,2,4


## Calculate the Win/Loss % for the Home team.

In [13]:
def calc_percentage(x, y):
    return 0 if y == 0 else (x / y)

In [14]:
if (use_parsed_data == False) & (calc_win_loss == True):
    
    # Create a dataframe for the results - same size as dataset
    data = pd.DataFrame(
        index=range(0,len(games)), 
        columns=[
            'totalGamesHome', 'totalWinsHome', 'homeGamesHome', 'homeWinsHome', 'awayGamesHome', 'awayWinsHome',
            'percentageTotalWinHome', 'percentageHomeWinHome', 'percentageAwayWinHome',
            'totalGamesAway', 'totalWinsAway', 'homeGamesAway', 'homeWinsAway', 'awayGamesAway', 'awayWinsAway',
            'percentageTotalWinAway', 'percentageHomeWinAway', 'percentageAwayWinAway'
        ]
    )

    # Iterate through each game.
    for index, game in games.iterrows():

        # Home Team
        # ---------
        
        historicGames = games[(games['season'] == game.season) & (games['gamesPlayedHome'] < game.gamesPlayedHome) & ((games['teamHomeCode'] == game.teamHomeCode) | (games['teamAwayCode'] == game.teamHomeCode))].sort_values(by='gamesPlayedHome', ascending=True)
       
        homeGamesHome = len(historicGames[(historicGames['teamHomeCode'] == game['teamHomeCode'])])        
        homeWinsHome = len(historicGames[(historicGames['teamHomeCode'] == game['teamHomeCode']) & (games["scoreAway"] < games["scoreHome"])])
        
        awayGamesHome = len(historicGames[(historicGames['teamAwayCode'] == game['teamHomeCode'])])        
        awayWinsHome = len(historicGames[(historicGames['teamAwayCode'] == game['teamHomeCode']) & (games["scoreAway"] > games["scoreHome"])])

        #totalGames = len(historicGames);
        #totalWins = len(
        #    historicGames[
        #        (historicGames['teamHomeCode'] == game['teamHomeCode']) & (games["scoreAway"] < games["scoreHome"]) |
        #        (historicGames['teamAwayCode'] == game['teamHomeCode']) & (games["scoreAway"] > games["scoreHome"])
        #    ]
        #)
        
        totalGamesHome = homeGamesHome + awayGamesHome
        totalWinsHome = homeWinsHome + awayWinsHome
        
        percentageTotalWinHome = round(calc_percentage(totalWinsHome, totalGamesHome), 3)
        percentageHomeWinHome = round(calc_percentage(homeWinsHome, homeGamesHome), 3)
        percentageAwayWinHome = round(calc_percentage(awayWinsHome, awayGamesHome), 3)
        
        # print('HOME', game['gamesPlayedHome'], 'TG', totalGamesHome, 'W', totalWinsHome, 'HGP', homeGamesHome, 'HW', homeWinsHome, 'AGP', awayGamesHome, 'AW', awayWinsHome)
        # print('HOME', 'W%', percentageTotalWinHome, 'HW%', percentageHomeWinHome, 'AW%', percentageAwayWinHome)
        
        # Away Team
        # ---------
        
        historicGames = games[(games['season'] == game.season) & (games['gamesPlayedAway'] < game.gamesPlayedAway) & ((games['teamAwayCode'] == game.teamAwayCode) | (games['teamHomeCode'] == game.teamAwayCode))].sort_values(by='gamesPlayedAway', ascending=True)
       
        homeGamesAway = len(historicGames[(historicGames['teamHomeCode'] == game['teamAwayCode'])])        
        homeWinsAway = len(historicGames[(historicGames['teamHomeCode'] == game['teamAwayCode']) & (games["scoreAway"] < games["scoreHome"])])
        
        awayGamesAway = len(historicGames[(historicGames['teamAwayCode'] == game['teamAwayCode'])])        
        awayWinsAway = len(historicGames[(historicGames['teamAwayCode'] == game['teamAwayCode']) & (games["scoreAway"] > games["scoreHome"])])

        #totalGames = len(historicGames);
        #totalWins = len(
        #    historicGames[
        #        (historicGames['teamHomeCode'] == game['teamHomeCode']) & (games["scoreAway"] < games["scoreHome"]) |
        #        (historicGames['teamAwayCode'] == game['teamHomeCode']) & (games["scoreAway"] > games["scoreHome"])
        #    ]
        #)
        
        totalGamesAway = homeGamesAway + awayGamesAway
        totalWinsAway = homeWinsAway + awayWinsAway
        
        percentageTotalWinAway = round(calc_percentage(totalWinsAway, totalGamesAway), 3)
        percentageHomeWinAway = round(calc_percentage(homeWinsAway, homeGamesAway), 3)
        percentageAwayWinAway = round(calc_percentage(awayWinsAway, awayGamesAway), 3)
        
        # print('AWAY', game['gamesPlayedAway'], 'TG', totalGamesAway, 'W', totalWinsAway, 'HGP', homeGamesAway, 'HW', homeWinsAway, 'AGP', awayGamesAway, 'AW', awayWinsAway)
        # print('AWAY', 'W%', percentageTotalWinAway, 'HW%', percentageHomeWinAway, 'AW%', percentageAwayWinAway)
        
        # ---------    
            
        # Update the row with the history
        data.loc[index] = [
            totalGamesHome, totalWinsHome, homeGamesHome, homeWinsHome, awayGamesHome, awayWinsHome,
            percentageTotalWinHome, percentageHomeWinHome, percentageAwayWinHome,
            totalGamesAway, totalWinsAway, homeGamesAway, homeWinsAway, awayGamesAway, awayWinsAway,
            percentageTotalWinAway, percentageHomeWinAway, percentageAwayWinAway
        ]

    # Add results to the dataset
    if (calc_win_loss_write_all == True):
        games = pd.merge(games, data, left_index=True, right_index=True)
        games[['teamHomeCode', 'teamAwayCode', 'percentageTotalWinHome', 'percentageHomeWinHome', 'percentageAwayWinHome', 'percentageTotalWinAway', 'percentageHomeWinAway', 'percentageAwayWinAway']].sample(5)
    else:
        games = pd.merge(games, data[['percentageHomeWinHome', 'percentageAwayWinAway']], left_index=True, right_index=True)
        games[['teamHomeCode', 'teamAwayCode', 'percentageHomeWinHome', 'percentageAwayWinAway']].sample(5)

/Users/jkentjnr/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/jkentjnr/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:26: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/jkentjnr/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:52: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/jkentjnr/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:55: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


### Calculate the results for the last 5 games for the Home team.

In [15]:
if (use_parsed_data == False) & (calc_last_5_games == True):
    
    # Create a dataframe for the results - same size as dataset
    data = pd.DataFrame(index=range(0,len(games)), columns=['lastGame1WinHome', 'lastGame1AtHomeHome', 'lastGame2WinHome', 'lastGame2AtHomeHome', 'lastGame3WinHome', 'lastGame3AtHomeHome', 'lastGame4WinHome', 'lastGame4AtHomeHome', 'lastGame5WinHome', 'lastGame5AtHomeHome'])

    # Iterate through each game where the teams have played at least 5 games.
    for index, game in games.iterrows():

        # Get the last five games for the team.
        last5games = games[(games['season'] == game.season) & (games['gamesPlayedHome'] < game.gamesPlayedHome) & ((games['teamAwayCode'] == game.teamHomeCode) | (games['teamHomeCode'] == game.teamHomeCode))].sort_values(by='gamesPlayedHome', ascending=False).head(5)

        if len(last5games) == 5:
            lastGame1WinHome = last5games["scoreAway"].iloc[0] < last5games["scoreHome"].iloc[0]
            lastGame2WinHome = last5games["scoreAway"].iloc[1] < last5games["scoreHome"].iloc[1]
            lastGame3WinHome = last5games["scoreAway"].iloc[2] < last5games["scoreHome"].iloc[2]
            lastGame4WinHome = last5games["scoreAway"].iloc[3] < last5games["scoreHome"].iloc[3]
            lastGame5WinHome = last5games["scoreAway"].iloc[4] < last5games["scoreHome"].iloc[4]
            #print(lastGame1WinHome, lastGame2WinHome, lastGame3WinHome, lastGame4WinHome, lastGame5WinHome)

            lastGame1AtHomeHome = last5games["teamHomeCode"].iloc[0] == game.teamHomeCode
            lastGame2AtHomeHome = last5games["teamHomeCode"].iloc[1] == game.teamHomeCode
            lastGame3AtHomeHome = last5games["teamHomeCode"].iloc[2] == game.teamHomeCode
            lastGame4AtHomeHome = last5games["teamHomeCode"].iloc[3] == game.teamHomeCode
            lastGame5AtHomeHome = last5games["teamHomeCode"].iloc[4] == game.teamHomeCode
            #print(lastGame1AtHomeHome, lastGame2AtHomeHome, lastGame3AtHomeHome, lastGame4AtHomeHome, lastGame5AtHomeHome)

            # Update the row with the history
            data.loc[index] = [lastGame1WinHome, lastGame1AtHomeHome, lastGame2WinHome, lastGame2AtHomeHome, lastGame3WinHome, lastGame3AtHomeHome, lastGame4WinHome, lastGame4AtHomeHome, lastGame5WinHome, lastGame5AtHomeHome]

    # Convert types 
    data['lastGame1WinHome'] = data['lastGame1WinHome'].astype('bool')
    data['lastGame2WinHome'] = data['lastGame2WinHome'].astype('bool')
    data['lastGame3WinHome'] = data['lastGame3WinHome'].astype('bool')
    data['lastGame4WinHome'] = data['lastGame4WinHome'].astype('bool')
    data['lastGame5WinHome'] = data['lastGame5WinHome'].astype('bool')
    data['lastGame1AtHomeHome'] = data['lastGame1AtHomeHome'].astype('bool')
    data['lastGame2AtHomeHome'] = data['lastGame2AtHomeHome'].astype('bool')
    data['lastGame3AtHomeHome'] = data['lastGame3AtHomeHome'].astype('bool')
    data['lastGame4AtHomeHome'] = data['lastGame4AtHomeHome'].astype('bool')
    data['lastGame5AtHomeHome'] = data['lastGame5AtHomeHome'].astype('bool')

    # Add results to the dataset
    games = pd.merge(games, data, left_index=True, right_index=True)
    
    games[['teamHomeCode', 'teamAwayCode', 'lastGame1WinHome', 'lastGame1AtHomeHome', 'lastGame2WinHome', 'lastGame2AtHomeHome', 'lastGame3WinHome', 'lastGame3AtHomeHome', 'lastGame4WinHome', 'lastGame4AtHomeHome', 'lastGame5WinHome', 'lastGame5AtHomeHome']].sample(5)

### Calculate the results for the last 5 games for the Away team.

In [16]:
if (use_parsed_data == False) & (calc_last_5_games == True):
    
    # Create a dataframe for the results - same size as dataset
    data = pd.DataFrame(index=range(0,len(games)), columns=['lastGame1WinAway', 'lastGame1AtHomeAway', 'lastGame2WinAway', 'lastGame2AtHomeAway', 'lastGame3WinAway', 'lastGame3AtHomeAway', 'lastGame4WinAway', 'lastGame4AtHomeAway', 'lastGame5WinAway', 'lastGame5AtHomeAway'])

    # Iterate through each game where the teams have played at least 5 games.
    for index, game in games.iterrows():

        # Get the last five games for the team.
        last5games = games[(games['season'] == game.season) & (games['gamesPlayedAway'] < game.gamesPlayedAway) & ((games['teamAwayCode'] == game.teamAwayCode) | (games['teamHomeCode'] == game.teamAwayCode))].sort_values(by='gamesPlayedAway', ascending=False).head(5)

        if len(last5games) == 5:
            lastGame1WinAway = last5games["scoreAway"].iloc[0] < last5games["scoreHome"].iloc[0]
            lastGame2WinAway = last5games["scoreAway"].iloc[1] < last5games["scoreHome"].iloc[1]
            lastGame3WinAway = last5games["scoreAway"].iloc[2] < last5games["scoreHome"].iloc[2]
            lastGame4WinAway = last5games["scoreAway"].iloc[3] < last5games["scoreHome"].iloc[3]
            lastGame5WinAway = last5games["scoreAway"].iloc[4] < last5games["scoreHome"].iloc[4]
            #print(lastGame1WinAway, lastGame2WinAway, lastGame3WinAway, lastGame4WinAway, lastGame5WinAway)

            lastGame1AtHomeAway = last5games["teamAwayCode"].iloc[0] == game.teamAwayCode
            lastGame2AtHomeAway = last5games["teamAwayCode"].iloc[1] == game.teamAwayCode
            lastGame3AtHomeAway = last5games["teamAwayCode"].iloc[2] == game.teamAwayCode
            lastGame4AtHomeAway = last5games["teamAwayCode"].iloc[3] == game.teamAwayCode
            lastGame5AtHomeAway = last5games["teamAwayCode"].iloc[4] == game.teamAwayCode
            #print(lastGame1AtHomeAway, lastGame2AtHomeAway, lastGame3AtHomeAway, lastGame4AtHomeAway, lastGame5AtHomeAway)

            # Update the row with the history
            data.loc[index] = [lastGame1WinAway, lastGame1AtHomeAway, lastGame2WinAway, lastGame2AtHomeAway, lastGame3WinAway, lastGame3AtHomeAway, lastGame4WinAway, lastGame4AtHomeAway, lastGame5WinAway, lastGame5AtHomeAway]

    # Convert types         
    data['lastGame1WinAway'] = data['lastGame1WinAway'].astype('bool')
    data['lastGame2WinAway'] = data['lastGame2WinAway'].astype('bool')
    data['lastGame3WinAway'] = data['lastGame3WinAway'].astype('bool')
    data['lastGame4WinAway'] = data['lastGame4WinAway'].astype('bool')
    data['lastGame5WinAway'] = data['lastGame5WinAway'].astype('bool')
    data['lastGame1AtHomeAway'] = data['lastGame1AtHomeAway'].astype('bool')
    data['lastGame2AtHomeAway'] = data['lastGame2AtHomeAway'].astype('bool')
    data['lastGame3AtHomeAway'] = data['lastGame3AtHomeAway'].astype('bool')
    data['lastGame4AtHomeAway'] = data['lastGame4AtHomeAway'].astype('bool')
    data['lastGame5AtHomeAway'] = data['lastGame5AtHomeAway'].astype('bool')

    # Add results to the dataset
    games = pd.merge(games, data, left_index=True, right_index=True)
    
    games[['teamHomeCode', 'teamAwayCode', 'lastGame1WinHome', 'lastGame1AtHomeHome', 'lastGame2WinHome', 'lastGame2AtHomeHome', 'lastGame3WinHome', 'lastGame3AtHomeHome', 'lastGame4WinHome', 'lastGame4AtHomeHome', 'lastGame5WinHome', 'lastGame5AtHomeHome']].sample(5)

### Calculate the average shooting % for the last 5 games

In [17]:
if (use_parsed_data == False) & (calc_ratings == True):
    
    # Create a dataframe for the results - same size as dataset
    data = pd.DataFrame(index=range(0,len(games)), columns=[
        'statsHome.seasonTotalPossessions', 'statsHome.last5TotalPossessions',
        'statsHome.seasonTotalPoints', 'statsHome.last5TotalPoints',
        'statsHome.seasonOffensiveRating', 'statsHome.last5OffensiveRating',
        'statsHome.seasonDefensiveRating', 'statsHome.last5DefensiveRating',
        'statsAway.seasonTotalPossessions', 'statsAway.last5TotalPossessions',
        'statsAway.seasonTotalPoints', 'statsAway.last5TotalPoints',
        'statsAway.seasonOffensiveRating', 'statsAway.last5OffensiveRating',
        'statsAway.seasonDefensiveRating', 'statsAway.last5DefensiveRating'
    ])

    # Iterate through each game where the teams have played at least 5 games.
    for index, game in games.iterrows():

        # Get all previous games - Home
        allgames = games[(games['season'] == game.season) & (games['gamesPlayedHome'] < game.gamesPlayedHome) & ((games['teamAwayCode'] == game.teamHomeCode) | (games['teamHomeCode'] == game.teamHomeCode))].sort_values(by='gamesPlayedHome', ascending=False)

        totalPossessionsHome = allgames['statsHome.FgAtt'].sum() - allgames['statsHome.OffReb'].sum() + allgames['statsHome.Tov'].sum() + (0.4 * allgames['statsHome.FtAtt'].sum());
        last5PossessionsHome = allgames['statsHome.FgAtt'].head(5).sum() - allgames['statsHome.OffReb'].head(5).sum() + allgames['statsHome.Tov'].head(5).sum() + (0.4 * allgames['statsHome.FtAtt'].head(5).sum());

        totalPointsScoredHome = allgames['statsHome.Pts'].sum()
        last5PointsScoredHome = allgames['statsHome.Pts'].head(5).sum()

        totalOffensiveRatingHome = round(calc_percentage(totalPointsScoredHome, totalPossessionsHome) * 100, 2)
        last5OffensiveRatingHome = round(calc_percentage(last5PointsScoredHome, last5PossessionsHome) * 100, 2)

        totalPointsAgainstHome = allgames['statsHome.PtsAgainst'].sum()
        last5PointsAgainstHome = allgames['statsHome.PtsAgainst'].head(5).sum()

        totalDefensiveRatingHome = round(calc_percentage(totalPointsAgainstHome, totalPossessionsHome) * 100, 2)
        last5DefensiveRatingHome = round(calc_percentage(last5PointsAgainstHome, last5PossessionsHome) * 100, 2)
        
        # Get all previous games - Away
        allgames = games[(games['season'] == game.season) & (games['gamesPlayedAway'] < game.gamesPlayedAway) & ((games['teamAwayCode'] == game.teamAwayCode) | (games['teamHomeCode'] == game.teamAwayCode))].sort_values(by='gamesPlayedAway', ascending=False)

        totalPossessionsAway = allgames['statsHome.FgAtt'].sum() - allgames['statsHome.OffReb'].sum() + allgames['statsHome.Tov'].sum() + (0.4 * allgames['statsHome.FtAtt'].sum());
        last5PossessionsAway = allgames['statsHome.FgAtt'].head(5).sum() - allgames['statsHome.OffReb'].head(5).sum() + allgames['statsHome.Tov'].head(5).sum() + (0.4 * allgames['statsHome.FtAtt'].head(5).sum());

        totalPointsScoredAway = allgames['statsHome.Pts'].sum()
        last5PointsScoredAway = allgames['statsHome.Pts'].head(5).sum()

        totalOffensiveRatingAway = round(calc_percentage(totalPointsScoredAway, totalPossessionsAway) * 100, 2)
        last5OffensiveRatingAway = round(calc_percentage(last5PointsScoredAway, last5PossessionsAway) * 100, 2)

        totalPointsAgainstAway = allgames['statsHome.PtsAgainst'].sum()
        last5PointsAgainstAway = allgames['statsHome.PtsAgainst'].head(5).sum()

        totalDefensiveRatingAway = round(calc_percentage(totalPointsAgainstAway, totalPossessionsAway) * 100, 2)
        last5DefensiveRatingAway = round(calc_percentage(last5PointsAgainstAway, last5PossessionsAway) * 100, 2)

        data.loc[index] = [
            totalPossessionsHome, last5PossessionsHome, 
            totalPointsScoredHome, last5PointsScoredHome, 
            totalOffensiveRatingHome, last5OffensiveRatingHome,
            totalDefensiveRatingHome, last5DefensiveRatingHome,
            totalPossessionsAway, last5PossessionsAway, 
            totalPointsScoredAway, last5PointsScoredAway, 
            totalOffensiveRatingAway, last5OffensiveRatingAway,
            totalDefensiveRatingAway, last5DefensiveRatingAway
        ]

        # Update the row with the history
        # data.loc[index] = [lastGame1WinAway, lastGame1AtHomeAway, lastGame2WinAway, lastGame2AtHomeAway, lastGame3WinAway, lastGame3AtHomeAway, lastGame4WinAway, lastGame4AtHomeAway, lastGame5WinAway, lastGame5AtHomeAway]

    data.sample(5)

    # Convert types
    # data['lastGame1WinAway'] = data['lastGame1WinAway'].astype('bool')

    # Add results to the dataset
    games = pd.merge(games, data, left_index=True, right_index=True)

    #games[['teamHomeCode', 'teamAwayCode', 'lastGame1WinHome', 'lastGame1AtHomeHome', 'lastGame2WinHome', 'lastGame2AtHomeHome', 'lastGame3WinHome', 'lastGame3AtHomeHome', 'lastGame4WinHome', 'lastGame4AtHomeHome', 'lastGame5WinHome', 'lastGame5AtHomeHome']].sample(5)


In [18]:
# test entry for stadium distance calc

coords_1 = (37.750267, -122.202853)
coords_2 = (34.04303865743706, -118.26711416244507)

geopy.distance.vincenty(coords_1, coords_2).miles


337.6783477035334

#### Save data to CSV to save on processing

In [19]:
if use_parsed_data == False:
    games.to_csv(output_parsed_data_filename, encoding='utf-8')

## Data Cleanse
Look at data types

In [20]:
games.dtypes

id                                    int64
location                             object
oddsBet365Away                      float64
oddsBet365Home                      float64
scoreAway                             uint8
scoreHome                             uint8
scoreQuarters                        object
season                                int64
statsAway.Ast                        object
statsAway.Blk                        object
statsAway.BlkAgainst                 object
statsAway.DefReb                     object
statsAway.Ejections                  object
statsAway.Fg2PtAtt                   object
statsAway.Fg2PtMade                  object
statsAway.Fg2PtPct                   object
statsAway.Fg3PtAtt                   object
statsAway.Fg3PtMade                  object
statsAway.Fg3PtPct                   object
statsAway.FgAtt                       uint8
statsAway.FgMade                     object
statsAway.FgPct                      object
statsAway.FoulFlag1             

Transform the Time column into a datetime object.

In [21]:
games["time"] = pd.to_datetime(games["time"])

### Convert the home and away teams to one-hot encoding 

In [22]:
if use_team_hot_encoding == True:
    homeTeam = pd.get_dummies(games["teamHomeCode"], prefix='team')
    awayTeam = pd.get_dummies(games["teamAwayCode"], prefix='team')

    homeTeam = homeTeam.replace({0:np.nan})
    awayTeam = awayTeam.replace({0:np.nan})

    teams = homeTeam.fillna(awayTeam).fillna(0).astype('bool')
    # teams[['team_NYK', 'team_CLE', 'team_ATL', 'team_BOS']].head(2)
    
    games = pd.concat([games, teams], axis=1)
    
    games[['teamHomeCode', 'teamAwayCode', 'team_NYK', 'team_CLE']].head(2)

In [23]:
if use_team_home_away_hot_encoding == True:
    homeTeam = pd.get_dummies(games["teamHomeCode"], prefix='hometeam').astype('bool')
    awayTeam = pd.get_dummies(games["teamAwayCode"], prefix='awayteam').astype('bool')

    games = pd.concat([games, homeTeam, awayTeam], axis=1)
    
    games[['teamHomeCode', 'teamAwayCode', 'hometeam_NYK', 'hometeam_CLE', 'awayteam_NYK', 'awayteam_CLE']].head(2)

Drop the id fields for teams.

In [24]:
if (use_team_hot_encoding == True) | (use_team_home_away_hot_encoding == True) | (use_team_keys == False):
    games = games.drop(["teamAwayId", "teamHomeId"], axis=1)

In [25]:
games = games.drop(["teamAwayCode", "teamHomeCode"], axis=1)

In [26]:
games = games.drop(["location", "scoreQuarters", "time"], axis=1)

In [27]:
games = games.drop(statsColumns, axis=1)

In [28]:
#gamesQuarters = json_normalize(data=d['games'], record_path='scoreQuarters', meta=['id'])
#gamesQuarters.head(10)

In [29]:
games["homeWin"] = games["scoreHome"] > games["scoreAway"]

In [30]:
# Remove games scores ... not possible to predict the future, if you already know it :)
games = games.drop(["scoreAway", "scoreHome"], axis=1)

In [31]:
# Remove any games that have less than a 5 game history.
if calc_last_5_games == True:
    games = games[(pd.notnull(games['lastGame1WinHome'])) & (pd.notnull(games['lastGame1AtHomeAway']))]

In [32]:
# Fill any NaN stats with 0
games.fillna(0, inplace=True)
#games['statsHome.Ejections'].fillna(0, inplace=True)
#games['statsAway.Ejections'].fillna(0, inplace=True)

In [33]:
# Show any NaN
games[games.isnull().T.any().T]

,id,oddsBet365Away,oddsBet365Home,season,gamesPlayedHome,gamesPlayedAway,totalGamesHome,totalWinsHome,homeGamesHome,homeWinsHome,...,statsHome.last5DefensiveRating,statsAway.seasonTotalPossessions,statsAway.last5TotalPossessions,statsAway.seasonTotalPoints,statsAway.last5TotalPoints,statsAway.seasonOffensiveRating,statsAway.last5OffensiveRating,statsAway.seasonDefensiveRating,statsAway.last5DefensiveRating,homeWin


In [34]:
games.sample(3)

,id,oddsBet365Away,oddsBet365Home,season,gamesPlayedHome,gamesPlayedAway,totalGamesHome,totalWinsHome,homeGamesHome,homeWinsHome,...,statsHome.last5DefensiveRating,statsAway.seasonTotalPossessions,statsAway.last5TotalPossessions,statsAway.seasonTotalPoints,statsAway.last5TotalPoints,statsAway.seasonOffensiveRating,statsAway.last5OffensiveRating,statsAway.seasonDefensiveRating,statsAway.last5DefensiveRating,homeWin
1196,32376,1.47,2.75,2015,24,25,21,11,9,6,...,115.99,2310.6,463.4,2609,555,112.91,119.77,109.32,115.24,True
1927,34641,1.40,3.10,2016,14,15,13,3,4,0,...,116.01,1293.0,468.6,1405,521,108.66,111.18,108.04,97.31,True
352,31528,2.50,1.58,2015,64,77,64,39,33,24,...,109.26,6709.6,432.6,7730,478,115.21,110.49,112.81,109.57,True


In [35]:
msk = np.random.rand(len(games)) < 0.8
msk

train = games[msk]
test = games[~msk]

Y_train = train["homeWin"]
X_train = train.drop(["id","homeWin"], axis=1)

#
X_test  = test.drop(["id", "homeWin"], axis=1).copy()
X_train.shape, Y_train.shape, X_test.shape

((1963, 59), (1963,), (497, 59))

In [36]:
# Reset the index
test.index = range(len(test.index))

In [37]:
X_test.sample(5)

,oddsBet365Away,oddsBet365Home,season,gamesPlayedHome,gamesPlayedAway,totalGamesHome,totalWinsHome,homeGamesHome,homeWinsHome,awayGamesHome,...,statsHome.seasonDefensiveRating,statsHome.last5DefensiveRating,statsAway.seasonTotalPossessions,statsAway.last5TotalPossessions,statsAway.seasonTotalPoints,statsAway.last5TotalPoints,statsAway.seasonOffensiveRating,statsAway.last5OffensiveRating,statsAway.seasonDefensiveRating,statsAway.last5DefensiveRating
1724,2.35,1.64,2016,69,67,69,37,34,20,35,...,110.67,114.63,6165.6,444.2,7195,549,116.70,123.59,108.94,124.04
167,3.40,1.33,2015,63,49,63,30,31,15,32,...,115.50,109.66,4165.2,440.4,4970,533,119.32,121.03,112.24,113.53
2050,2.45,1.60,2016,7,5,5,2,3,1,2,...,117.58,117.58,294.2,294.2,332,332,112.85,112.85,110.47,110.47
1594,5.25,1.18,2016,77,76,77,49,37,27,40,...,114.08,116.81,6852.8,452.2,8112,543,118.37,120.08,114.95,113.00
1537,1.43,2.90,2016,81,41,80,42,40,23,40,...,114.54,114.52,3816.2,452.2,4321,553,113.23,122.29,110.21,109.46


### Logistic Regression

In [38]:
logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
Y_pred = logreg.predict(X_test)
acc_log = round(logreg.score(X_train, Y_train) * 100, 2)
acc_log

69.540000000000006

In [39]:
coeff_df = pd.DataFrame(games.columns.delete(0))
coeff_df.columns = ['Feature']
coeff_df["Correlation"] = pd.Series(logreg.coef_[0])

coeff_df.sort_values(by='Correlation', ascending=False)

,Feature,Correlation
0,oddsBet365Away,0.250399
12,percentageHomeWinHome,0.167778
36,lastGame2AtHomeAway,0.117717
11,percentageTotalWinHome,0.114426
24,lastGame1AtHomeHome,0.073209
10,awayWinsHome,0.051645
13,percentageAwayWinHome,0.046575
30,lastGame4AtHomeHome,0.045066
25,lastGame2WinHome,0.036276
16,homeGamesAway,0.033088


In [40]:
result = pd.concat([test, pd.DataFrame(Y_pred, columns=['forecast'])], axis=1)
total = len(result)
correct = len(result[(result['homeWin'] == result['forecast'])])
incorrect = len(result[(result['homeWin'] != result['forecast'])])
correct_log = round((correct / total) * 100, 2)

print(correct_log, total, correct, incorrect)

67.61 497 336 161


### Support Vector Machines

In [41]:
svc = SVC()
svc.fit(X_train, Y_train)
Y_pred = svc.predict(X_test)
acc_svc = round(svc.score(X_train, Y_train) * 100, 2)
acc_svc

99.689999999999998

In [42]:
result = pd.concat([test, pd.DataFrame(Y_pred, columns=['forecast'])], axis=1)
total = len(result)
correct = len(result[(result['homeWin'] == result['forecast'])])
incorrect = len(result[(result['homeWin'] != result['forecast'])])
correct_svc = round((correct / total) * 100, 2)

print(correct_svc, total, correct, incorrect)

60.36 497 300 197


### k-Nearest Neighbors

In [43]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, Y_train)
Y_pred = knn.predict(X_test)
acc_knn = round(knn.score(X_train, Y_train) * 100, 2)
acc_knn

76.409999999999997

In [44]:
result = pd.concat([test, pd.DataFrame(Y_pred, columns=['forecast'])], axis=1)
total = len(result)
correct = len(result[(result['homeWin'] == result['forecast'])])
incorrect = len(result[(result['homeWin'] != result['forecast'])])
correct_knn = round((correct / total) * 100, 2)

print(correct_knn, total, correct, incorrect)

54.73 497 272 225


### Gaussian Naive Bayes

In [45]:
gaussian = GaussianNB()
gaussian.fit(X_train, Y_train)
Y_pred = gaussian.predict(X_test)
acc_gaussian = round(gaussian.score(X_train, Y_train) * 100, 2)
acc_gaussian

66.480000000000004

In [46]:
result = pd.concat([test, pd.DataFrame(Y_pred, columns=['forecast'])], axis=1)
total = len(result)
correct = len(result[(result['homeWin'] == result['forecast'])])
incorrect = len(result[(result['homeWin'] != result['forecast'])])
correct_gaussian = round((correct / total) * 100, 2)

print(correct_gaussian, total, correct, incorrect)

64.99 497 323 174


### Perceptron

In [47]:
perceptron = Perceptron(max_iter = 5)
perceptron.fit(X_train, Y_train)
Y_pred = perceptron.predict(X_test)
acc_perceptron = round(perceptron.score(X_train, Y_train) * 100, 2)
acc_perceptron

58.07

In [48]:
result = pd.concat([test, pd.DataFrame(Y_pred, columns=['forecast'])], axis=1)
total = len(result)
correct = len(result[(result['homeWin'] == result['forecast'])])
incorrect = len(result[(result['homeWin'] != result['forecast'])])
correct_perceptron = round((correct / total) * 100, 2)

print(correct_perceptron, total, correct, incorrect)

61.37 497 305 192


### Linear SVC

In [49]:
linear_svc = LinearSVC()
linear_svc.fit(X_train, Y_train)
Y_pred = linear_svc.predict(X_test)
acc_linear_svc = round(linear_svc.score(X_train, Y_train) * 100, 2)
acc_linear_svc

61.840000000000003

In [50]:
result = pd.concat([test, pd.DataFrame(Y_pred, columns=['forecast'])], axis=1)
total = len(result)
correct = len(result[(result['homeWin'] == result['forecast'])])
incorrect = len(result[(result['homeWin'] != result['forecast'])])
correct_linear_svc = round((correct / total) * 100, 2)

print(correct_linear_svc, total, correct, incorrect)

63.98 497 318 179


### Stochastic Gradient Descent

In [51]:
sgd = SGDClassifier(max_iter = 5)
sgd.fit(X_train, Y_train)
Y_pred = sgd.predict(X_test)
acc_sgd = round(sgd.score(X_train, Y_train) * 100, 2)
acc_sgd

42.539999999999999

In [52]:
result = pd.concat([test, pd.DataFrame(Y_pred, columns=['forecast'])], axis=1)
total = len(result)
correct = len(result[(result['homeWin'] == result['forecast'])])
incorrect = len(result[(result['homeWin'] != result['forecast'])])
correct_sgd = round((correct / total) * 100, 2)

print(correct_sgd, total, correct, incorrect)

39.64 497 197 300


### Decision Tree

In [53]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
Y_pred = decision_tree.predict(X_test)
acc_decision_tree = round(decision_tree.score(X_train, Y_train) * 100, 2)
acc_decision_tree

100.0

In [54]:
result = pd.concat([test, pd.DataFrame(Y_pred, columns=['forecast'])], axis=1)
total = len(result)
correct = len(result[(result['homeWin'] == result['forecast'])])
incorrect = len(result[(result['homeWin'] != result['forecast'])])
correct_decision_tree = round((correct / total) * 100, 2)

print(correct_decision_tree, total, correct, incorrect)

57.34 497 285 212


### Random Forest

In [55]:
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)
acc_random_forest = round(random_forest.score(X_train, Y_train) * 100, 2)
acc_random_forest

100.0

In [56]:
result = pd.concat([test, pd.DataFrame(Y_pred, columns=['forecast'])], axis=1)
total = len(result)
correct = len(result[(result['homeWin'] == result['forecast'])])
incorrect = len(result[(result['homeWin'] != result['forecast'])])
correct_random_forest = round((correct / total) * 100, 2)

print(correct_random_forest, len(Y_pred), total, correct, incorrect)

66.0 497 497 328 169


## Model Evaluation Results

In [57]:
models = pd.DataFrame({
    'Score': [acc_svc, acc_knn, acc_log, 
              acc_random_forest, acc_gaussian, acc_perceptron, 
              acc_sgd, acc_linear_svc, acc_decision_tree],
    'Correct': [correct_svc, correct_knn, correct_log, 
              correct_random_forest, correct_gaussian, correct_perceptron, 
              correct_sgd, correct_linear_svc, correct_decision_tree],
    'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
              'Random Forest', 'Naive Bayes', 'Perceptron', 
              'Stochastic Gradient Decent', 'Linear SVC', 
              'Decision Tree']})
models.sort_values(by='Correct', ascending=False)

,Correct,Model,Score
2,67.61,Logistic Regression,69.54
3,66.00,Random Forest,100.00
4,64.99,Naive Bayes,66.48
7,63.98,Linear SVC,61.84
5,61.37,Perceptron,58.07
0,60.36,Support Vector Machines,99.69
8,57.34,Decision Tree,100.00
1,54.73,KNN,76.41
6,39.64,Stochastic Gradient Decent,42.54


In [58]:
result[(result['homeWin'] != result['forecast'])].sample(10)

,id,oddsBet365Away,oddsBet365Home,season,gamesPlayedHome,gamesPlayedAway,totalGamesHome,totalWinsHome,homeGamesHome,homeWinsHome,...,statsAway.seasonTotalPossessions,statsAway.last5TotalPossessions,statsAway.seasonTotalPoints,statsAway.last5TotalPoints,statsAway.seasonOffensiveRating,statsAway.last5OffensiveRating,statsAway.seasonDefensiveRating,statsAway.last5DefensiveRating,homeWin,forecast
253,33970,2.40,1.62,2016,57,56,54,28,22,13,...,5226.4,483.8,6124,595,117.17,122.98,111.51,108.31,True,False
481,35089,1.74,2.15,2016,24,24,21,3,10,1,...,2244.8,451.2,2605,547,116.05,121.23,114.67,114.14,True,False
59,31497,2.55,1.57,2015,79,79,79,42,39,25,...,6876.2,426.8,7926,498,115.27,116.68,112.71,110.59,False,True
385,34631,2.30,1.66,2016,14,15,13,8,8,5,...,1158.2,454.4,1349,512,116.47,112.68,116.47,115.32,False,True
341,34437,1.90,1.90,2016,69,71,66,25,33,16,...,6090.0,441.8,7096,510,116.52,115.44,114.07,121.77,True,False
67,31532,2.90,1.43,2015,78,79,77,44,39,26,...,6845.6,429.2,7798,476,113.91,110.90,109.06,114.63,False,True
84,31632,3.60,1.31,2015,74,71,73,28,37,15,...,6225.4,440.2,7331,526,117.76,119.49,112.86,123.58,False,True
14,31285,2.10,1.76,2015,55,55,57,20,28,15,...,4807.2,452.4,5516,469,114.74,103.67,111.58,105.00,True,False
25,31337,3.00,1.41,2015,51,51,53,20,27,15,...,4434.2,455.8,5032,500,113.48,109.70,112.87,115.62,False,True
150,31981,1.43,2.90,2015,8,9,8,4,4,3,...,593.2,418.2,730,520,123.06,124.34,118.51,121.95,True,False


In [59]:
result[(result['id'] == 33946) | (result['id'] == 34601)]

,id,oddsBet365Away,oddsBet365Home,season,gamesPlayedHome,gamesPlayedAway,totalGamesHome,totalWinsHome,homeGamesHome,homeWinsHome,...,statsAway.seasonTotalPossessions,statsAway.last5TotalPossessions,statsAway.seasonTotalPoints,statsAway.last5TotalPoints,statsAway.seasonOffensiveRating,statsAway.last5OffensiveRating,statsAway.seasonDefensiveRating,statsAway.last5DefensiveRating,homeWin,forecast


In [60]:
games[(games['id'] == 33946) | (games['id'] == 34601)]

,id,oddsBet365Away,oddsBet365Home,season,gamesPlayedHome,gamesPlayedAway,totalGamesHome,totalWinsHome,homeGamesHome,homeWinsHome,...,statsHome.last5DefensiveRating,statsAway.seasonTotalPossessions,statsAway.last5TotalPossessions,statsAway.seasonTotalPoints,statsAway.last5TotalPoints,statsAway.seasonOffensiveRating,statsAway.last5OffensiveRating,statsAway.seasonDefensiveRating,statsAway.last5DefensiveRating,homeWin
1235,33946,10.50,1.06,2016,58,58,57,36,28,20,...,119.07,5469.8,479.2,6214,505,113.61,105.38,113.42,108.51,True
1887,34601,1.07,9.50,2016,14,16,10,3,5,1,...,109.07,1447.0,484.0,1745,589,120.59,121.69,114.86,110.95,False
